In [ ]:
# Cell 01: Setup (Drive + fixed tree + clean clone + snapshot)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, shutil, subprocess, time, json

BASE = "/content/drive/MyDrive/UCRNLP_YueDong_Screening"
REPO_URL = "https://github.com/erfanshayegani/Jailbreak-In-Pieces"
REPO_DIR = f"{BASE}/01_repo/Jailbreak-In-Pieces"

def sh(cmd):
    print("\n$", cmd)
    out = subprocess.check_output(["bash","-lc", cmd], stderr=subprocess.STDOUT, text=True)
    print(out[:20000])
    return out

# 1) Create required Drive tree (exact)
folders = [
    f"{BASE}/01_repo",
    f"{BASE}/02_notebooks",
    f"{BASE}/03_outputs",
    f"{BASE}/03_outputs/baseline",
    f"{BASE}/03_outputs/attack_run_1",
    f"{BASE}/03_outputs/attack_run_2_ablation",
    f"{BASE}/03_outputs/figures",
    f"{BASE}/03_outputs/tables",
    f"{BASE}/03_outputs/logs",
    f"{BASE}/04_report",
    f"{BASE}/04_report/assets",
    f"{BASE}/05_submission",
]
for p in folders:
    os.makedirs(p, exist_ok=True)

# 2) Clean clone (wipe + depth=1)
if os.path.isdir(REPO_DIR):
    shutil.rmtree(REPO_DIR, ignore_errors=True)
sh(f"git clone --depth 1 '{REPO_URL}' '{REPO_DIR}'")

# 3) Snapshot: tree + commit + README header
sh(f"ls -la '{BASE}'")
sh(f"ls -la '{BASE}/03_outputs'")
sh(f"cd '{REPO_DIR}' && git rev-parse --short HEAD && git status -sb")
sh(f"cd '{REPO_DIR}' && find . -maxdepth 2 -type f | sed 's|^./||' | sort | head -n 200")
sh(f"cd '{REPO_DIR}' && sed -n '1,80p' README.md")

# 4) Log snapshot to Drive
snap = {
    "time": time.ctime(),
    "base": BASE,
    "repo_dir": REPO_DIR,
}
with open(f"{BASE}/03_outputs/logs/cell01_snapshot.json", "w") as f:
    json.dump(snap, f, indent=2)
print("\nWrote:", f"{BASE}/03_outputs/logs/cell01_snapshot.json")

Mounted at /content/drive

$ git clone --depth 1 'https://github.com/erfanshayegani/Jailbreak-In-Pieces' '/content/drive/MyDrive/UCRNLP_YueDong_Screening/01_repo/Jailbreak-In-Pieces'
Cloning into '/content/drive/MyDrive/UCRNLP_YueDong_Screening/01_repo/Jailbreak-In-Pieces'...


$ ls -la '/content/drive/MyDrive/UCRNLP_YueDong_Screening'
total 20
drwx------ 3 root root 4096 Jan 17 01:50 01_repo
drwx------ 2 root root 4096 Jan 16 06:25 02_notebooks
drwx------ 8 root root 4096 Jan 16 06:25 03_outputs
drwx------ 3 root root 4096 Jan 16 06:25 04_report
drwx------ 2 root root 4096 Jan 16 06:25 05_submission


$ ls -la '/content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs'
total 24
drwx------ 2 root root 4096 Jan 16 06:25 attack_run_1
drwx------ 2 root root 4096 Jan 16 06:25 attack_run_2_ablation
drwx------ 2 root root 4096 Jan 16 06:25 baseline
drwx------ 2 root root 4096 Jan 16 06:25 figures
drwx------ 2 root root 4096 Jan 16 06:25 logs
drwx------ 2 root root 4096 Jan 16 06:25 tables



In [ ]:
# Cell 02: GPU + installs + env log
import os, subprocess, sys, time

BASE = "/content/drive/MyDrive/UCRNLP_YueDong_Screening"
REPO_DIR = f"{BASE}/01_repo/Jailbreak-In-Pieces"
LOG_DIR = f"{BASE}/03_outputs/logs"
os.makedirs(LOG_DIR, exist_ok=True)

def sh(cmd):
    print("\n$", cmd)
    out = subprocess.check_output(["bash","-lc", cmd], stderr=subprocess.STDOUT, text=True)
    print(out[:20000])
    return out

sh("nvidia-smi || true")
sh("python -V")
sh("pip -V")
sh("pip install -q --upgrade pip")

# README requirements
sh("pip install -q pillow numpy matplotlib sentence-transformers transformers")

# torch/torchvision (Colab usually has them; install only if missing)
try:
    import torch, torchvision
    print("torch OK:", torch.__version__, "cuda:", torch.cuda.is_available())
    print("torchvision OK:", torchvision.__version__)
except Exception as e:
    print("torch/torchvision missing or broken -> reinstalling")
    sh("pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121")

# Final import check + log
import torch, torchvision, transformers
from sentence_transformers import SentenceTransformer
ver_path = f"{LOG_DIR}/env_versions.txt"
with open(ver_path, "w") as f:
    f.write(f"time: {time.ctime()}\n")
    f.write(f"python: {sys.version}\n")
    f.write(f"torch: {torch.__version__} cuda:{torch.cuda.is_available()}\n")
    f.write(f"torchvision: {torchvision.__version__}\n")
    f.write(f"transformers: {transformers.__version__}\n")
print("Wrote:", ver_path)

sh(f"ls -la '{REPO_DIR}/src'")


$ nvidia-smi || true
Sat Jan 17 01:50:32 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------------

'total 10\n-rw------- 1 root root 6107 Jan 17 01:50 adv_image.py\n-rw------- 1 root root 3580 Jan 17 01:50 utils.py\n'

In [ ]:
# Cell 03 (REPLACE): Extract supported knobs (no full file dump)
import subprocess

REPO_DIR = "/content/drive/MyDrive/UCRNLP_YueDong_Screening/01_repo/Jailbreak-In-Pieces"

def sh(cmd):
    print("\n$", cmd)
    out = subprocess.check_output(["bash","-lc", cmd], stderr=subprocess.STDOUT, text=True)
    print(out[:20000])
    return out

# Only show the relevant hyperparameter lines (no full source print)
sh(f"cd '{REPO_DIR}' && nl -ba src/adv_image.py | egrep -n 'model_id|optimizer|lr=|num_epochs|clamp\\(|PairwiseDistance' | head -n 200")


$ cd '/content/drive/MyDrive/UCRNLP_YueDong_Screening/01_repo/Jailbreak-In-Pieces' && nl -ba src/adv_image.py | egrep -n 'model_id|optimizer|lr=|num_epochs|clamp\(|PairwiseDistance' | head -n 200
67:    67	    # model_id = "openai/clip-vit-base-patch32"
68:    68	    # model_id = "openai/clip-vit-large-patch14"
69:    69	    model_id = "openai/clip-vit-large-patch14-336"
72:    72	    tokenizer = CLIPTokenizerFast.from_pretrained(model_id) # for processing text
73:    73	    processor = CLIPProcessor.from_pretrained(model_id) # for processing image
74:    74	    model = CLIPModel.from_pretrained(model_id).to(device)# for giving us the embeddings
76:    76	    imgproc = CLIPImageProcessor.from_pretrained(model_id)
89:    89	    pdist = torch.nn.PairwiseDistance(p=2)
101:   101	    # Set up optimizer
102:   102	    optimizer = optim.Adam([random_img], lr=0.1) # tune the learning rate
105:   105	    num_epochs = 5000 
107:   107	    for epoch in range(num_epochs):
108:   108	        opti

'67:    67\t    # model_id = "openai/clip-vit-base-patch32"\n68:    68\t    # model_id = "openai/clip-vit-large-patch14"\n69:    69\t    model_id = "openai/clip-vit-large-patch14-336"\n72:    72\t    tokenizer = CLIPTokenizerFast.from_pretrained(model_id) # for processing text\n73:    73\t    processor = CLIPProcessor.from_pretrained(model_id) # for processing image\n74:    74\t    model = CLIPModel.from_pretrained(model_id).to(device)# for giving us the embeddings\n76:    76\t    imgproc = CLIPImageProcessor.from_pretrained(model_id)\n89:    89\t    pdist = torch.nn.PairwiseDistance(p=2)\n101:   101\t    # Set up optimizer\n102:   102\t    optimizer = optim.Adam([random_img], lr=0.1) # tune the learning rate\n105:   105\t    num_epochs = 5000 \n107:   107\t    for epoch in range(num_epochs):\n108:   108\t        optimizer.zero_grad()\n113:   113\t        optimizer.step()\n114:   114\t        random_img.data = torch.clamp(random_img.data, 0.0, 1.0)\n129:   129\t    tensor = (tensor * 2

In [ ]:
# Cell 04 (REPLACE): Build SAFE sample set for baseline + targets (no repo "JBPieces_logo.png")
import os, shutil, subprocess
from PIL import Image, ImageDraw

BASE = "/content/drive/MyDrive/UCRNLP_YueDong_Screening"
REPO_DIR = f"{BASE}/01_repo/Jailbreak-In-Pieces"
SAMPLES_DIR = f"{BASE}/03_outputs/baseline/samples"
os.makedirs(SAMPLES_DIR, exist_ok=True)

def sh(cmd):
    print("\n$", cmd)
    out = subprocess.check_output(["bash","-lc", cmd], stderr=subprocess.STDOUT, text=True)
    print(out[:20000])
    return out

# 0) Clean sample directory to avoid stale/incorrect files
for fn in os.listdir(SAMPLES_DIR):
    if fn.lower().endswith((".png", ".jpg", ".jpeg")):
        os.remove(os.path.join(SAMPLES_DIR, fn))

# 1) Copy ONLY benign repo images
safe_repo_imgs = [
    f"{REPO_DIR}/images/nature.jpeg",
    f"{REPO_DIR}/images/white.jpeg",
]
for p in safe_repo_imgs:
    assert os.path.exists(p), f"Missing: {p}"
    shutil.copy2(p, SAMPLES_DIR)

# 2) Generate SAFE synthetic images (solid + checkerboard + circles)
W = H = 336

solid = Image.new("RGB", (W, H), (240, 240, 240))
solid.save(f"{SAMPLES_DIR}/solid_gray.png")

chk = Image.new("RGB", (W, H), (255, 255, 255))
d = ImageDraw.Draw(chk)
step = 24
for y in range(0, H, step):
    for x in range(0, W, step):
        if (x//step + y//step) % 2 == 0:
            d.rectangle([x, y, x+step-1, y+step-1], fill=(30, 30, 30))
chk.save(f"{SAMPLES_DIR}/checkerboard.png")

cir = Image.new("RGB", (W, H), (255, 255, 255))
dc = ImageDraw.Draw(cir)
for r in [20, 50, 85, 120, 150]:
    dc.ellipse([W//2-r, H//2-r, W//2+r, H//2+r], outline=(0,0,0), width=4)
cir.save(f"{SAMPLES_DIR}/circles.png")

# 3) Quick verification previews (optional but clean)
def save_preview(src, outname):
    im = Image.open(src).convert("RGB")
    im.thumbnail((700,700))
    out = f"{BASE}/03_outputs/figures/{outname}"
    os.makedirs(os.path.dirname(out), exist_ok=True)
    im.save(out)
    print("Saved preview:", out, "size=", im.size)

save_preview(f"{SAMPLES_DIR}/circles.png", "VERIFY_circles_target_preview.png")

# 4) List samples
sh(f"ls -la '{SAMPLES_DIR}'")

Saved preview: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/figures/VERIFY_circles_target_preview.png size= (336, 336)

$ ls -la '/content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/baseline/samples'
total 275
-rw------- 1 root root   1246 Jan 17 01:50 checkerboard.png
-rw------- 1 root root   4907 Jan 17 01:50 circles.png
-rw------- 1 root root 235508 Jan 17 01:50 nature.jpeg
-rw------- 1 root root   1053 Jan 17 01:50 solid_gray.png
-rw------- 1 root root  37197 Jan 17 01:50 white.jpeg



'total 275\n-rw------- 1 root root   1246 Jan 17 01:50 checkerboard.png\n-rw------- 1 root root   4907 Jan 17 01:50 circles.png\n-rw------- 1 root root 235508 Jan 17 01:50 nature.jpeg\n-rw------- 1 root root   1053 Jan 17 01:50 solid_gray.png\n-rw------- 1 root root  37197 Jan 17 01:50 white.jpeg\n'

In [ ]:
# Cell 05 (REPLACE): Baseline (CLIP similarity on 5 safe samples) + save CSV + heatmap + meta
import os, json, time
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from PIL import Image
from transformers import CLIPModel, CLIPImageProcessor

BASE="/content/drive/MyDrive/UCRNLP_YueDong_Screening"
SAMPLES=f"{BASE}/03_outputs/baseline/samples"
OUT_BASE=f"{BASE}/03_outputs/baseline"
TAB=f"{BASE}/03_outputs/tables"
FIG=f"{BASE}/03_outputs/figures"
os.makedirs(OUT_BASE, exist_ok=True)
os.makedirs(TAB, exist_ok=True)
os.makedirs(FIG, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID="openai/clip-vit-large-patch14-336"

imgproc = CLIPImageProcessor.from_pretrained(MODEL_ID)
imgproc.do_normalize = False
model = CLIPModel.from_pretrained(MODEL_ID).to(device)
model.eval()

files = ["checkerboard.png","nature.jpeg","solid_gray.png","white.jpeg","circles.png"]
paths = [f"{SAMPLES}/{f}" for f in files]
for p in paths:
    assert os.path.exists(p), p

def embed_image(path):
    im = Image.open(path).convert("RGB")
    x = imgproc(images=im, return_tensors="pt")["pixel_values"].to(device)
    with torch.no_grad():
        emb = model.get_image_features(x)
    emb = emb / emb.norm(dim=-1, keepdim=True)
    return emb.squeeze(0).detach().cpu().numpy()

embs = np.stack([embed_image(p) for p in paths], axis=0)
sim = embs @ embs.T

df = pd.DataFrame(sim, index=files, columns=files)
csv_path = f"{TAB}/baseline_clip_cosine_similarity.csv"
df.to_csv(csv_path)

plt.figure(figsize=(7,6))
plt.imshow(sim, vmin=-1, vmax=1)
plt.xticks(range(len(files)), files, rotation=45, ha="right")
plt.yticks(range(len(files)), files)
plt.colorbar(label="cosine similarity")
plt.tight_layout()
fig_path = f"{FIG}/baseline_clip_similarity_heatmap.png"
plt.savefig(fig_path, dpi=200)
plt.close()

n = sim.shape[0]
mask = ~np.eye(n, dtype=bool)
avg_off = float(sim[mask].mean())
max_off = float(sim[mask].max())
min_off = float(sim[mask].min())

meta = {
    "run": "baseline_clip_similarity",
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "device": device,
    "model_id": MODEL_ID,
    "samples": paths,
    "n_samples": n,
    "avg_offdiag_cosine": avg_off,
    "max_offdiag_cosine": max_off,
    "min_offdiag_cosine": min_off,
    "artifacts": {"csv": csv_path, "heatmap": fig_path},
}
meta_path = f"{OUT_BASE}/baseline_run_meta.json"
json.dump(meta, open(meta_path,"w"), indent=2)

print("Saved:", csv_path)
print("Saved:", fig_path)
print("Saved:", meta_path)
print(df.round(4))
print("\nBaseline summary:", {"n_samples": n, "avg_offdiag_cosine": avg_off, "max_offdiag_cosine": max_off, "min_offdiag_cosine": min_off})

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Saved: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/tables/baseline_clip_cosine_similarity.csv
Saved: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/figures/baseline_clip_similarity_heatmap.png
Saved: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/baseline/baseline_run_meta.json
                  checkerboard.png  nature.jpeg  solid_gray.png  white.jpeg  \
checkerboard.png            1.0000       0.6635          0.7452      0.7725   
nature.jpeg                 0.6635       1.0000          0.7402      0.7821   
solid_gray.png              0.7452       0.7402          1.0000      0.9580   
white.jpeg                  0.7725       0.7821          0.9580      1.0000   
circles.png                 0.7331       0.6293          0.6613      0.7006   

                  circles.png  
checkerboard.png       0.7331  
nature.jpeg            0.6293  
solid_gray.png         0.6613  
white.jpeg             0.7006  
circles.png            1.0000  

Baseline summa

In [ ]:
# Cell 06 (REPLACE): Attack Run 1 — clean logging (every 50) + save artifacts (safe targets only)
import os, json, time, random, pickle
import numpy as np
import torch
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
from transformers import CLIPModel, CLIPImageProcessor

BASE="/content/drive/MyDrive/UCRNLP_YueDong_Screening"
SAMPLES_DIR=f"{BASE}/03_outputs/baseline/samples"
OUT_DIR=f"{BASE}/03_outputs/attack_run_1"
FIG_DIR=f"{BASE}/03_outputs/figures"
LOG_DIR=f"{BASE}/03_outputs/logs"
os.makedirs(OUT_DIR, exist_ok=True); os.makedirs(FIG_DIR, exist_ok=True); os.makedirs(LOG_DIR, exist_ok=True)

device="cuda" if torch.cuda.is_available() else "cpu"
seed=42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

MODEL_ID="openai/clip-vit-large-patch14-336"
imgproc=CLIPImageProcessor.from_pretrained(MODEL_ID); imgproc.do_normalize=False
model=CLIPModel.from_pretrained(MODEL_ID).to(device); model.eval()
pdist=torch.nn.PairwiseDistance(p=2)

targets=[
    ("checkerboard", f"{SAMPLES_DIR}/checkerboard.png"),
    ("nature",       f"{SAMPLES_DIR}/nature.jpeg"),
    ("solid_gray",   f"{SAMPLES_DIR}/solid_gray.png"),
    ("circles",      f"{SAMPLES_DIR}/circles.png"),
]
init_path=f"{SAMPLES_DIR}/white.jpeg"
for _,p in targets: assert os.path.exists(p), p
assert os.path.exists(init_path), init_path

def tensor_to_pil(x01):
    t=x01.detach().cpu().clamp(0,1)[0]
    arr=(t*255).to(torch.uint8).permute(1,2,0).numpy()
    return Image.fromarray(arr)

def save_loss_curve(loss_list, path, title):
    plt.figure()
    plt.plot(loss_list)
    plt.title(title)
    plt.xlabel("epoch")
    plt.ylabel("L2 distance")
    plt.tight_layout()
    plt.savefig(path, dpi=200)
    plt.close()

def save_panel(init_img, target_img, adv_img, path):
    W,H=336,336
    panel=Image.new("RGB",(W*3,H),(255,255,255))
    panel.paste(init_img.resize((W,H)),(0,0))
    panel.paste(target_img.resize((W,H)),(W,0))
    panel.paste(adv_img.resize((W,H)),(W*2,0))
    panel.save(path)

lr=0.05
epochs=800
checkpoints=set([1] + list(range(50, epochs+1, 50)) + [epochs])

runner_log={
    "run":"attack_run_1",
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "device": device, "seed": seed, "model_id": MODEL_ID,
    "init_image": init_path,
    "targets":[{"tag":t,"path":p} for t,p in targets],
    "results":[]
}

print(f"[RUN1 START] {time.strftime('%c')} device={device} seed={seed}")

for tag, target_path in targets:
    meta_path=f"{OUT_DIR}/attack1_{tag}_meta.json"
    adv_path=f"{OUT_DIR}/attack1_{tag}_adv.png"
    loss_pkl=f"{OUT_DIR}/attack1_{tag}_loss.pkl"
    panel_png=f"{FIG_DIR}/attack1_{tag}_panel.png"
    curve_png=f"{FIG_DIR}/attack1_{tag}_loss_curve.png"

    if os.path.exists(meta_path) and os.path.exists(panel_png) and os.path.exists(adv_path):
        m=json.load(open(meta_path))
        print(f"[SKIP] {tag} already done -> {meta_path}")
        runner_log["results"].append(m)
        continue

    print(f"\n[START attack1] tag={tag} target={os.path.basename(target_path)} lr={lr} epochs={epochs}")

    random_img=imgproc(images=Image.open(init_path).convert("RGB"), return_tensors="pt")["pixel_values"].to(device)
    random_img.requires_grad=True
    optimizer=optim.Adam([random_img], lr=lr)

    target_tensor=imgproc(images=Image.open(target_path).convert("RGB"), return_tensors="pt")["pixel_values"].to(device)
    with torch.no_grad():
        target_emb=model.get_image_features(target_tensor)

    loss_list=[]
    t0=time.time()
    for ep in range(1, epochs+1):
        optimizer.zero_grad(set_to_none=True)
        adv_emb=model.get_image_features(random_img)
        loss=pdist(adv_emb, target_emb.detach())
        loss.backward()
        optimizer.step()
        random_img.data=torch.clamp(random_img.data, 0.0, 1.0)
        loss_list.append(float(loss.item()))
        if ep in checkpoints:
            print(f"  epoch={ep:>4}/{epochs} loss={loss.item():.6f} elapsed_sec={time.time()-t0:.1f}")

    adv_pil=tensor_to_pil(random_img)
    adv_pil.save(adv_path)
    with open(loss_pkl,"wb") as f: pickle.dump(loss_list,f)
    save_loss_curve(loss_list, curve_png, f"Run1 ({tag}) loss (epochs={epochs})")

    init_pil=Image.open(init_path).convert("RGB")
    target_pil=Image.open(target_path).convert("RGB")
    save_panel(init_pil, target_pil, adv_pil, panel_png)

    meta={
        "run":"attack_run_1","tag":tag,
        "target_path":target_path,"init_path":init_path,
        "model_id":MODEL_ID,"device":device,"seed":seed,
        "lr":lr,"epochs":epochs,"final_loss":float(loss_list[-1]),
        "artifacts":{"adv_image":adv_path,"loss_pkl":loss_pkl,"loss_curve":curve_png,"panel":panel_png},
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    }
    json.dump(meta, open(meta_path,"w"), indent=2)
    runner_log["results"].append(meta)
    print(f"[DONE attack1] tag={tag} final_loss={loss_list[-1]:.6f}\n  meta: {meta_path}")

log_path=f"{LOG_DIR}/attack_run_1_runner_log.json"
json.dump(runner_log, open(log_path,"w"), indent=2)
print(f"\n[RUN1 END] {time.strftime('%c')}\nRunner log: {log_path}")

[RUN1 START] Sat Jan 17 01:51:16 2026 device=cuda seed=42
[SKIP] checkerboard already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_1/attack1_checkerboard_meta.json
[SKIP] nature already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_1/attack1_nature_meta.json
[SKIP] solid_gray already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_1/attack1_solid_gray_meta.json
[SKIP] circles already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_1/attack1_circles_meta.json

[RUN1 END] Sat Jan 17 01:51:18 2026
Runner log: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/logs/attack_run_1_runner_log.json


In [ ]:
# Cell 07: Attack Run 1 (cached) — safe targets only
import os, json, time, random, pickle
import numpy as np
import torch
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
from transformers import CLIPModel, CLIPImageProcessor

BASE="/content/drive/MyDrive/UCRNLP_YueDong_Screening"
SAMPLES_DIR=f"{BASE}/03_outputs/baseline/samples"
OUT_DIR=f"{BASE}/03_outputs/attack_run_1"
FIG_DIR=f"{BASE}/03_outputs/figures"
LOG_DIR=f"{BASE}/03_outputs/logs"
os.makedirs(OUT_DIR, exist_ok=True); os.makedirs(FIG_DIR, exist_ok=True); os.makedirs(LOG_DIR, exist_ok=True)

device="cuda" if torch.cuda.is_available() else "cpu"
seed=42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

MODEL_ID="openai/clip-vit-large-patch14-336"
imgproc=CLIPImageProcessor.from_pretrained(MODEL_ID); imgproc.do_normalize=False
model=CLIPModel.from_pretrained(MODEL_ID).to(device); model.eval()
pdist=torch.nn.PairwiseDistance(p=2)

targets=[
    ("checkerboard", f"{SAMPLES_DIR}/checkerboard.png"),
    ("nature",       f"{SAMPLES_DIR}/nature.jpeg"),
    ("solid_gray",   f"{SAMPLES_DIR}/solid_gray.png"),
    ("circles",      f"{SAMPLES_DIR}/circles.png"),
]
init_path=f"{SAMPLES_DIR}/white.jpeg"
for _,p in targets: assert os.path.exists(p), p
assert os.path.exists(init_path), init_path

def tensor_to_pil(x01):
    t=x01.detach().cpu().clamp(0,1)[0]
    arr=(t*255).to(torch.uint8).permute(1,2,0).numpy()
    return Image.fromarray(arr)

def save_loss_curve(loss_list, path, title):
    plt.figure(); plt.plot(loss_list)
    plt.title(title); plt.xlabel("epoch"); plt.ylabel("L2 distance")
    plt.tight_layout(); plt.savefig(path, dpi=200); plt.close()

def save_panel(init_img, target_img, adv_img, path):
    W,H=336,336
    panel=Image.new("RGB",(W*3,H),(255,255,255))
    panel.paste(init_img.resize((W,H)),(0,0))
    panel.paste(target_img.resize((W,H)),(W,0))
    panel.paste(adv_img.resize((W,H)),(W*2,0))
    panel.save(path)

lr=0.05
epochs=800

runner_log={
    "run":"attack_run_1",
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "device": device, "seed": seed, "model_id": MODEL_ID,
    "init_image": init_path,
    "targets":[{"tag":t,"path":p} for t,p in targets],
    "results":[]
}

print(f"[RUN1 START] {time.strftime('%c')} device={device} seed={seed}")

for tag, target_path in targets:
    meta_path=f"{OUT_DIR}/attack1_{tag}_meta.json"
    adv_path=f"{OUT_DIR}/attack1_{tag}_adv.png"
    loss_pkl=f"{OUT_DIR}/attack1_{tag}_loss.pkl"
    panel_png=f"{FIG_DIR}/attack1_{tag}_panel.png"
    curve_png=f"{FIG_DIR}/attack1_{tag}_loss_curve.png"

    if os.path.exists(meta_path) and os.path.exists(panel_png) and os.path.exists(adv_path):
        m=json.load(open(meta_path))
        print(f"[SKIP] {tag} already done -> {meta_path}")
        runner_log["results"].append(m)
        continue

    print(f"\n[START] tag={tag} target={os.path.basename(target_path)} lr={lr} epochs={epochs}")

    random_img=imgproc(images=Image.open(init_path).convert("RGB"), return_tensors="pt")["pixel_values"].to(device)
    random_img.requires_grad=True
    optimizer=optim.Adam([random_img], lr=lr)

    target_tensor=imgproc(images=Image.open(target_path).convert("RGB"), return_tensors="pt")["pixel_values"].to(device)
    with torch.no_grad():
        target_emb=model.get_image_features(target_tensor)

    loss_list=[]
    t0=time.time()
    for ep in range(1, epochs+1):
        optimizer.zero_grad(set_to_none=True)
        adv_emb=model.get_image_features(random_img)
        loss=pdist(adv_emb, target_emb.detach())
        loss.backward()
        optimizer.step()
        random_img.data=torch.clamp(random_img.data, 0.0, 1.0)
        loss_list.append(float(loss.item()))
        if ep in [1,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800]:
            print(f"  epoch={ep:>4}/{epochs} loss={loss.item():.6f} elapsed_sec={time.time()-t0:.1f}")

    adv_pil=tensor_to_pil(random_img)
    adv_pil.save(adv_path)
    with open(loss_pkl,"wb") as f: pickle.dump(loss_list,f)
    save_loss_curve(loss_list, curve_png, f"Run1 ({tag}) loss (epochs={epochs})")

    init_pil=Image.open(init_path).convert("RGB")
    target_pil=Image.open(target_path).convert("RGB")
    save_panel(init_pil, target_pil, adv_pil, panel_png)

    meta={
        "run":"attack_run_1","tag":tag,
        "target_path":target_path,"init_path":init_path,
        "model_id":MODEL_ID,"device":device,"seed":seed,
        "lr":lr,"epochs":epochs,"final_loss":float(loss_list[-1]),
        "artifacts":{"adv_image":adv_path,"loss_pkl":loss_pkl,"loss_curve":curve_png,"panel":panel_png},
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    }
    json.dump(meta, open(meta_path,"w"), indent=2)
    runner_log["results"].append(meta)
    print(f"[DONE] {tag} final_loss={loss_list[-1]:.6f}")

log_path=f"{LOG_DIR}/attack_run_1_runner_log.json"
json.dump(runner_log, open(log_path,"w"), indent=2)
print(f"\n[RUN1 END] {time.strftime('%c')}\nRunner log: {log_path}")


[RUN1 START] Sat Jan 17 01:51:20 2026 device=cuda seed=42
[SKIP] checkerboard already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_1/attack1_checkerboard_meta.json
[SKIP] nature already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_1/attack1_nature_meta.json
[SKIP] solid_gray already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_1/attack1_solid_gray_meta.json
[SKIP] circles already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_1/attack1_circles_meta.json

[RUN1 END] Sat Jan 17 01:51:20 2026
Runner log: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/logs/attack_run_1_runner_log.json


In [ ]:
# Cell 08 (REPLACE): Attack Run 2 (Ablation cached) — ONE knob changed: epochs=300
import os, json, time, random, pickle
import numpy as np
import torch
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
from transformers import CLIPModel, CLIPImageProcessor

BASE="/content/drive/MyDrive/UCRNLP_YueDong_Screening"
SAMPLES_DIR=f"{BASE}/03_outputs/baseline/samples"
OUT_DIR=f"{BASE}/03_outputs/attack_run_2_ablation"
FIG_DIR=f"{BASE}/03_outputs/figures"
LOG_DIR=f"{BASE}/03_outputs/logs"
os.makedirs(OUT_DIR, exist_ok=True); os.makedirs(FIG_DIR, exist_ok=True); os.makedirs(LOG_DIR, exist_ok=True)

device="cuda" if torch.cuda.is_available() else "cpu"
seed=42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

MODEL_ID="openai/clip-vit-large-patch14-336"
imgproc=CLIPImageProcessor.from_pretrained(MODEL_ID); imgproc.do_normalize=False
model=CLIPModel.from_pretrained(MODEL_ID).to(device); model.eval()
pdist=torch.nn.PairwiseDistance(p=2)

targets=[
    ("checkerboard", f"{SAMPLES_DIR}/checkerboard.png"),
    ("nature",       f"{SAMPLES_DIR}/nature.jpeg"),
    ("solid_gray",   f"{SAMPLES_DIR}/solid_gray.png"),
    ("circles",      f"{SAMPLES_DIR}/circles.png"),
]
init_path=f"{SAMPLES_DIR}/white.jpeg"
for _,p in targets: assert os.path.exists(p), p
assert os.path.exists(init_path), init_path

def tensor_to_pil(x01):
    t=x01.detach().cpu().clamp(0,1)[0]
    arr=(t*255).to(torch.uint8).permute(1,2,0).numpy()
    return Image.fromarray(arr)

def save_loss_curve(loss_list, path, title):
    plt.figure(); plt.plot(loss_list)
    plt.title(title); plt.xlabel("epoch"); plt.ylabel("L2 distance")
    plt.tight_layout(); plt.savefig(path, dpi=200); plt.close()

def save_panel(init_img, target_img, adv_img, path):
    W,H=336,336
    panel=Image.new("RGB",(W*3,H),(255,255,255))
    panel.paste(init_img.resize((W,H)),(0,0))
    panel.paste(target_img.resize((W,H)),(W,0))
    panel.paste(adv_img.resize((W,H)),(W*2,0))
    panel.save(path)

lr=0.05
epochs=300  # ONE knob changed

runner_log={
    "run":"attack_run_2_ablation",
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "device": device, "seed": seed, "model_id": MODEL_ID,
    "init_image": init_path,
    "epochs": epochs,
    "targets":[{"tag":t,"path":p} for t,p in targets],
    "results":[]
}

print(f"[ABLATION START] {time.strftime('%c')} device={device} seed={seed} epochs={epochs}")

for tag, target_path in targets:
    meta_path=f"{OUT_DIR}/ablation_{tag}_meta.json"
    adv_path=f"{OUT_DIR}/ablation_{tag}_adv.png"
    loss_pkl=f"{OUT_DIR}/ablation_{tag}_loss.pkl"
    panel_png=f"{FIG_DIR}/ablation_{tag}_panel.png"
    curve_png=f"{FIG_DIR}/ablation_{tag}_loss_curve.png"

    if os.path.exists(meta_path) and os.path.exists(panel_png) and os.path.exists(adv_path):
        m=json.load(open(meta_path))
        print(f"[SKIP] {tag} already done -> {meta_path}")
        runner_log["results"].append(m)
        continue

    print(f"\n[START] tag={tag} target={os.path.basename(target_path)} lr={lr} epochs={epochs}")

    random_img=imgproc(images=Image.open(init_path).convert("RGB"), return_tensors="pt")["pixel_values"].to(device)
    random_img.requires_grad=True
    optimizer=optim.Adam([random_img], lr=lr)

    target_tensor=imgproc(images=Image.open(target_path).convert("RGB"), return_tensors="pt")["pixel_values"].to(device)
    with torch.no_grad():
        target_emb=model.get_image_features(target_tensor)

    loss_list=[]
    t0=time.time()
    for ep in range(1, epochs+1):
        optimizer.zero_grad(set_to_none=True)
        adv_emb=model.get_image_features(random_img)
        loss=pdist(adv_emb, target_emb.detach())
        loss.backward()
        optimizer.step()
        random_img.data=torch.clamp(random_img.data, 0.0, 1.0)
        loss_list.append(float(loss.item()))
        if ep in [1,50,100,150,200,250,300]:
            print(f"  epoch={ep:>4}/{epochs} loss={loss.item():.6f} elapsed_sec={time.time()-t0:.1f}")

    adv_pil=tensor_to_pil(random_img)
    adv_pil.save(adv_path)
    with open(loss_pkl,"wb") as f: pickle.dump(loss_list,f)
    save_loss_curve(loss_list, curve_png, f"Ablation ({tag}) loss (epochs={epochs})")

    init_pil=Image.open(init_path).convert("RGB")
    target_pil=Image.open(target_path).convert("RGB")
    save_panel(init_pil, target_pil, adv_pil, panel_png)

    meta={
        "run":"attack_run_2_ablation","tag":tag,
        "target_path":target_path,"init_path":init_path,
        "model_id":MODEL_ID,"device":device,"seed":seed,
        "lr":lr,"epochs":epochs,"final_loss":float(loss_list[-1]),
        "artifacts":{"adv_image":adv_path,"loss_pkl":loss_pkl,"loss_curve":curve_png,"panel":panel_png},
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    }
    json.dump(meta, open(meta_path,"w"), indent=2)
    runner_log["results"].append(meta)
    print(f"[DONE] {tag} final_loss={loss_list[-1]:.6f}")

log_path=f"{LOG_DIR}/attack_run_2_ablation_runner_log.json"
json.dump(runner_log, open(log_path,"w"), indent=2)
print(f"\n[ABLATION END] {time.strftime('%c')}\nRunner log: {log_path}")

[ABLATION START] Sat Jan 17 01:51:23 2026 device=cuda seed=42 epochs=300
[SKIP] checkerboard already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_2_ablation/ablation_checkerboard_meta.json
[SKIP] nature already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_2_ablation/ablation_nature_meta.json
[SKIP] solid_gray already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_2_ablation/ablation_solid_gray_meta.json
[SKIP] circles already done -> /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/attack_run_2_ablation/ablation_circles_meta.json

[ABLATION END] Sat Jan 17 01:51:25 2026
Runner log: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/logs/attack_run_2_ablation_runner_log.json


In [ ]:
# Cell 09 (REPLACE): Build summary tables + figure selection for report (safe-only) — robust meta parsing
import os, json
import pandas as pd

BASE="/content/drive/MyDrive/UCRNLP_YueDong_Screening"
TAB=f"{BASE}/03_outputs/tables"
FIG=f"{BASE}/03_outputs/figures"
RUN1=f"{BASE}/03_outputs/attack_run_1"
ABL=f"{BASE}/03_outputs/attack_run_2_ablation"
os.makedirs(TAB, exist_ok=True)

SAFE_TAGS=set(["checkerboard","nature","solid_gray","circles"])

def load_meta(folder, prefix):
    rows=[]
    for fn in sorted(os.listdir(folder)):
        if not (fn.endswith("_meta.json") and fn.startswith(prefix)):
            continue
        p=os.path.join(folder, fn)
        m=json.load(open(p))

        tag = m.get("tag", None)
        if tag not in SAFE_TAGS:
            continue  # ignore old logo/other artifacts

        rows.append({
            "tag": tag,
            "epochs": int(m.get("epochs", -1)),
            "lr": float(m.get("lr", float("nan"))),
            "seed": int(m.get("seed", m.get("random_seed", 42))),  # default 42 if missing
            "final_loss": float(m.get("final_loss", float("nan"))),
            "panel": m.get("artifacts", {}).get("panel", m.get("panel_path", "")),
            "loss_curve": m.get("artifacts", {}).get("loss_curve", m.get("loss_curve", "")),
            "meta_path": p,
        })
    df=pd.DataFrame(rows)
    # keep latest per tag (in case old runs exist)
    if len(df)==0:
        return df
    df = df.sort_values("meta_path").drop_duplicates(subset=["tag"], keep="last")
    return df

df_run1 = load_meta(RUN1, "attack1_")
df_abl  = load_meta(ABL,  "ablation_")

assert set(df_run1["tag"])==SAFE_TAGS, f"Run1 tags mismatch: {set(df_run1['tag'])}"
assert set(df_abl["tag"])==SAFE_TAGS, f"Ablation tags mismatch: {set(df_abl['tag'])}"

# Baseline summary
baseline_meta = f"{BASE}/03_outputs/baseline/baseline_run_meta.json"
assert os.path.exists(baseline_meta), "Missing baseline_run_meta.json. Re-run baseline cell."
bm = json.load(open(baseline_meta))

df_baseline = pd.DataFrame([{
    "n_samples": bm.get("n_samples", 5),
    "avg_offdiag_cosine": bm.get("avg_offdiag_cosine", None),
    "max_offdiag_cosine": bm.get("max_offdiag_cosine", None),
    "min_offdiag_cosine": bm.get("min_offdiag_cosine", None),
}])

# Save tables
p_bas = f"{TAB}/baseline_summary.csv"
p_r1  = f"{TAB}/attack_run_1_summary.csv"
p_ab  = f"{TAB}/attack_run_2_ablation_summary.csv"

df_baseline.to_csv(p_bas, index=False)
df_run1.sort_values("final_loss").to_csv(p_r1, index=False)
df_abl.sort_values("final_loss").to_csv(p_ab, index=False)

print("Saved:", p_bas)
print("Saved:", p_r1)
print("Saved:", p_ab)

# Figure selection (qualitative: 4 panels from Run1)
qual_panels = {
    "A1_checkerboard": df_run1.loc[df_run1.tag=="checkerboard","panel"].item(),
    "A2_nature":       df_run1.loc[df_run1.tag=="nature","panel"].item(),
    "A3_solid_gray":   df_run1.loc[df_run1.tag=="solid_gray","panel"].item(),
    "A4_circles":      df_run1.loc[df_run1.tag=="circles","panel"].item(),
}

# Failure cases: worst final_loss in Run1 and worst in Ablation
run1_fail = df_run1.sort_values("final_loss", ascending=False).iloc[0]
abl_fail  = df_abl.sort_values("final_loss", ascending=False).iloc[0]

selection = {
    "qualitative_panels": qual_panels,
    "attack_run_1_failure": {"tag": run1_fail["tag"], "panel": run1_fail["panel"], "final_loss": float(run1_fail["final_loss"])},
    "ablation_failure":     {"tag": abl_fail["tag"],  "panel": abl_fail["panel"],  "final_loss": float(abl_fail["final_loss"])},
    "tables": {
        "baseline": p_bas,
        "attack_run_1": p_r1,
        "attack_run_2_ablation": p_ab
    }
}

sel_path = f"{TAB}/report_figure_selection.json"
json.dump(selection, open(sel_path,"w"), indent=2)
print("Saved:", sel_path)

print("\nRun1 best->worst:\n", df_run1[["tag","epochs","final_loss"]].sort_values("final_loss"))
print("\nAblation best->worst:\n", df_abl[["tag","epochs","final_loss"]].sort_values("final_loss"))
print("\nFailure cases picked:", selection["attack_run_1_failure"], selection["ablation_failure"])

Saved: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/tables/baseline_summary.csv
Saved: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/tables/attack_run_1_summary.csv
Saved: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/tables/attack_run_2_ablation_summary.csv
Saved: /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/tables/report_figure_selection.json

Run1 best->worst:
             tag  epochs  final_loss
3    solid_gray     800    0.138592
2        nature     800    0.223899
0  checkerboard     800    0.224061
1       circles     800    0.341626

Ablation best->worst:
             tag  epochs  final_loss
0  checkerboard     300    0.408771
3    solid_gray     300    0.429354
2        nature     300    0.437501
1       circles     300    0.469027

Failure cases picked: {'tag': 'circles', 'panel': '/content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/figures/attack1_circles_panel.png', 'final_loss': 0.341625839471817} {'tag': 'circle

In [12]:
# FINAL CHECK: verify submission files (IPYNB + DOCX) + required outputs
import os, glob

BASE="/content/drive/MyDrive/UCRNLP_YueDong_Screening"
SUB=f"{BASE}/05_submission"
OUT=f"{BASE}/03_outputs"
TAB=f"{OUT}/tables"
FIG=f"{OUT}/figures"
SAMPLES=f"{OUT}/baseline/samples"

need = [
    f"{SUB}/UCRNLP_JailbreakInPieces_Attack.ipynb",
    f"{SUB}/UCRNLP_JailbreakInPieces_Report.pdf",
    f"{OUT}/baseline/baseline_run_meta.json",
    f"{TAB}/baseline_summary.csv",
    f"{TAB}/attack_run_1_summary.csv",
    f"{TAB}/attack_run_2_ablation_summary.csv",
    f"{TAB}/report_figure_selection.json",
    f"{FIG}/baseline_clip_similarity_heatmap.png",
    f"{FIG}/attack1_checkerboard_panel.png",
    f"{FIG}/attack1_nature_panel.png",
    f"{FIG}/attack1_solid_gray_panel.png",
    f"{FIG}/attack1_circles_panel.png",
    f"{FIG}/ablation_checkerboard_panel.png",
    f"{FIG}/ablation_nature_panel.png",
    f"{FIG}/ablation_solid_gray_panel.png",
    f"{FIG}/ablation_circles_panel.png",
]

print("=== REQUIRED FILES ===")
ok_all=True
for p in need:
    ok=os.path.exists(p)
    ok_all = ok_all and ok
    print(("OK " if ok else "MISSING "), p)

print("\n=== SAFE SAMPLE SET (must NOT contain JBPieces_logo.png) ===")
sample_files=sorted([
    os.path.basename(x)
    for x in glob.glob(f"{SAMPLES}/*")
    if x.lower().endswith((".png",".jpg",".jpeg"))
])
print(sample_files)
if "JBPieces_logo.png" in sample_files:
    ok_all=False
    print("FAIL: JBPieces_logo.png present in samples.")
else:
    print("OK: no repo harmful 'logo' in samples.")

print("\n=== RESULT ===")
print("PASS" if ok_all else "FAIL")

=== REQUIRED FILES ===
OK  /content/drive/MyDrive/UCRNLP_YueDong_Screening/05_submission/UCRNLP_JailbreakInPieces_Attack.ipynb
OK  /content/drive/MyDrive/UCRNLP_YueDong_Screening/05_submission/UCRNLP_JailbreakInPieces_Report.pdf
OK  /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/baseline/baseline_run_meta.json
OK  /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/tables/baseline_summary.csv
OK  /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/tables/attack_run_1_summary.csv
OK  /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/tables/attack_run_2_ablation_summary.csv
OK  /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/tables/report_figure_selection.json
OK  /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/figures/baseline_clip_similarity_heatmap.png
OK  /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/figures/attack1_checkerboard_panel.png
OK  /content/drive/MyDrive/UCRNLP_YueDong_Screening/03_outputs/figures/attack